# Panel Regressions

This notebook illustrates several different panel data models and estimation methods (pooled OLS, fixed effects, the "between" estimator, first differences, etc)

## Loading Packages

In [1]:
using Dates, DelimitedFiles, Statistics, LinearAlgebra

include("jlFiles/printmat.jl")
include("jlFiles/printTable.jl")
include("jlFiles/NWFn.jl")
include("jlFiles/OlsFn.jl")
include("jlFiles/lagnFn.jl")
include("jlFiles/exciseFn.jl")

exciseFn (generic function with 1 method)

## Loading Data

In [2]:
x = readdlm("Data/nls_panelEd.txt",skipstart=1)    #clasical data set from Hill et al (2008)

NT = size(x,1)
c  = ones(NT)

T = 5                 #number of time periods
N = round(Int,NT/T)   #number of individuals         

(id,year,lnwage)   = (x[:,1],x[:,2],x[:,3])
(exper,exper2)     = (x[:,15],x[:,16])
(tenure,tenure2)   = (x[:,17],x[:,18])
(south,tradeunion) = (x[:,12],x[:,14])

id = round.(Int,id)    #id should be an integer (1,2,3,...)

println("T=$T and N=$N")

T=5 and N=716


## Creating Variables for the Regressions

The next cell creates a matrix $yx$ which has the dependent variable as the first column and the regressors as the remaining columns.

The first observations of the data set are tabulated. Notice the structure: the first 5 observations are for individual (`id`) 1 (period 1-5), the next 5 for individual 2. 

The subsequent cell makes a "within transformation" by creating 

$
yx^*_{it} = yx_{it} - \bar{yx}_{it}, 
$

where $\bar{yx}_{it}$ is a row vector with the averages of each column of $yx$ for individual $i$.

In [3]:
xNames = ["exper/100","exper^2/100","tenure/100","tenure^2/100","south","union"]
yx = [lnwage c exper/100 exper2/100 tenure/100 tenure2/100 south tradeunion]
K  = size(yx,2) - 1

printblue("The first lines of the data set:")
rowNames = [string("obs",i) for i=1:11]
printTable(Any[id[1:11] yx[1:11,1:3]],["id" "lnwage" "c" "exper/100"],rowNames)

The first lines of the data set:
             id    lnwage         c exper/100
obs1          1     1.808     1.000     0.077
obs2          1     1.863     1.000     0.086
obs3          1     1.789     1.000     0.102
obs4          1     1.847     1.000     0.122
obs5          1     1.856     1.000     0.136
obs6          2     1.281     1.000     0.076
obs7          2     1.516     1.000     0.084
obs8          2     1.930     1.000     0.104
obs9          2     1.919     1.000     0.120
obs10         2     2.201     1.000     0.132
obs11         3     1.815     1.000     0.114



In [4]:
id_uniq = unique(id)               #which id values are in data set
N       = length(id_uniq)

yxStar = fill(NaN,size(yx))          #individual de-meaning
yxbar  = fill(NaN,(N,1+K)) 
for i = 1:N                          #loop over individuals
    #local vv_i                      #only needed in REPL/script
    vv_i          = id .== id_uniq[i]                #locate rows in yx which refer to individual i
    yxbar[i,:]    = mean(yx[vv_i,:],dims=1)          #averages for individual i
    yxStar[vv_i,:] = yx[vv_i,:] .- yxbar[i:i,:]      #i:i to keep it a row vector
end

## Pooled OLS, FE, and Between Estimations

In [5]:
(b,res,yhat,Covb,R2,) = OlsFn(yx[:,1],yx[:,2:end])            #pooled OLS
xutLS = hcat(b,b./sqrt.(diag(Covb)))
xutLS = xutLS[2:end,:]


(b,res,yhat,Covb,R2,) = OlsFn(yxStar[:,1],yxStar[:,3:end])    #fixed effect
xutFE = hcat(b,b./sqrt.(diag(Covb))*sqrt(NT-N-2)/sqrt(NT-2))
s2_e  = sum(res.^2)/(NT-N-(K-1))              #needed for GLS (see below)               


(b,res,yhat,Covb,R2,) = OlsFn(yxbar[:,1],yxbar[:,2:end])      #between estimator
xutB = hcat(b,b./sqrt.(diag(Covb)))
xutB = xutB[2:end,:]
s2_u = max(0,sum(res.^2)/(N-K) - s2_e/T)      #needed for GLS

printblue("Point estimates")
printTable([xutLS[:,1] xutFE[:,1] xutB[:,1]],["Pooled" "FE" "Between"],xNames)

printblue("t-stats")
printTable([xutLS[:,2] xutFE[:,2] xutB[:,2]],["Pooled" "FE" "Between"],xNames)

Point estimates
                Pooled        FE   Between
exper/100        7.837     4.108    10.641
exper^2/100     -0.201    -0.041    -0.317
tenure/100       1.206     1.391     1.247
tenure^2/100    -0.024    -0.090    -0.016
south           -0.196    -0.016    -0.201
union            0.110     0.064     0.121

t-stats
                Pooled        FE   Between
exper/100        8.954     5.917     4.573
exper^2/100     -5.264    -1.466    -3.054
tenure/100       2.346     3.975     0.883
tenure^2/100    -0.828    -4.136    -0.198
south          -13.247    -0.367    -6.519
union            6.928     4.181     3.102



## First-Difference Model

To estimate the first-difference model, we first need to calculate the differences (over two time periods) for the *same individual*.

In the cell below, we call on the function `lagnFn` which lags the data once (as a default). For the first time period, the result is a NaN (as there are no earlier values). After the loop we locate and delete all rows that include some NaNs. This means that we will have only $T-1$ data points for each individual. Finally, we replace the zeros created 

In [6]:
yxStarΔ = fill(NaN,size(yx))
for i = 1:N                          #individual first-differencing, loop over individuals
    #local vv_i                      #only in REPL/scripts 
    vv_i            = id .== id_uniq[i]   #rows in yx which refer to individual i
    yxStarΔ[vv_i,:] = yx[vv_i,:] - lagnFn(yx[vv_i,:])   #yx[t] -yx[t-1]
end

yxStarΔ = exciseFn(yxStarΔ)          #cut out rows with NaNs
yxStarΔ[:,2] .= 1                    #constant  

println("size of yxStarΔ: ",size(yxStarΔ))

size of yxStarΔ: (2864, 8)


In [7]:
(b,res,yhat,Covb,R2,) = OlsFn(yxStarΔ[:,1],yxStarΔ[:,2:end])
xutΔ = hcat(b,b./sqrt.(diag(Covb)))
xutΔ = xutΔ[2:end,:]

printblue("1-st difference estimation:")
printTable(xutΔ,["Coef" "Std"],xNames)

1-st difference estimation:
                  Coef       Std
exper/100        3.548     2.277
exper^2/100     -0.045    -0.933
tenure/100       1.293     2.527
tenure^2/100    -0.083    -2.329
south           -0.024    -0.395
union            0.044     3.115



## GLS of Random Effects Model (extra)

In [8]:
ϑ = 1 - sqrt(s2_e)/sqrt(T*s2_u+s2_e)                       #GLS
yxStar_ϑ = fill(NaN,size(yx))
for i = 1:N
    #local vv_i              #only needed in REPL/script
    vv_i             = id .== id_uniq[i]
    yxStar_ϑ[vv_i,:] = yx[vv_i,:] .- ϑ*yxbar[i:i,:]       
end
(b,res,yhat,Covb,R2,) = OlsFn(yxStar_ϑ[:,1],yxStar_ϑ[:,2:end])
xutGLS = hcat(b,b./sqrt.(diag(Covb)))
xutGLS = xutGLS[2:end,:]

printblue("GLS:")
printTable(xutGLS,["Coef" "Std"],xNames)

GLS:
                  Coef       Std
exper/100        4.570     7.111
exper^2/100     -0.063    -2.387
tenure/100       1.380     4.032
tenure^2/100    -0.074    -3.575
south           -0.132    -5.255
union            0.075     5.611

